<h1> Ajay Basran </h1>
<h3> Date start: 2022-06-01 
<br>
<br> 
NHL Teams Stanley Cup Prediction Algorithm </h3>

**Necessary Libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
import math as m
import sys
from datetime import date

#OPG Side
import ssl

**Data Acquisition**

In [2]:
#Global Variables
headers = ['Team',
               'Games played',
               'Wins',
               'Losses',
               'Overtime Losses',
               'Points',
               'Point Percentage',
               'Goals For',
               'Goals Against',
               'Goal Differential',
               'Goals For per Game',
               'Goals Against per Game',
               'Goal Differential per Game'
#                'RPT_perc',
#                'RW',
#                'RgRec',
#                'RgPt_perc'
                ]

In [3]:
def generate_urls ():
    #URL Setup
    today = str(date.today()).split("-")
    url = "https://www.hockey-reference.com/boxscores/?year=" + today[0] + "&month=" + today[1] + "&day=" + today[2]
    return url

In [4]:
def response_request(url):
    try:
        response = requests.get(url)
    except ssl.SSLError:
        response = requests.get(url, verify = False)

    #use a parsing library to create a parsable object 
    soup = BeautifulSoup(response.text, "html.parser") # Create a soup object (Once)
    return(soup)

In [5]:
def extract_table (soup):
    
    teams = []
    team_stats = []
    team_names = []
    
    table_tag = soup.find('table', {'data-stat':"team_name", 'class': 'left '})
    # Collecting data
    for tr in soup.find_all('tr', table_tag):    

        # Find all data for each column
        #if th found -> columns[0].text.strip()
        columns = tr.find_all('td')
        column_headers = tr.find_all('th')
        #print(columns)
        #print()

        row_items = 11 #Stats in row that need to be targeted
        if (len(columns) < row_items):
            pass
        else:
            #Gathering Team Stats by parsing through td
            for td in columns:
                #Teams = columns[0].text.strip()
                GamesPlayed = int(columns[0].text.strip())
                Wins = int(columns[1].text.strip())
                Losses = int(columns[2].text.strip())
                OvertimeLosses = int(columns[3].text.strip())
                Points = int(columns[4].text.strip())
                Points_perc = Points/(GamesPlayed*2)
                GoalsFor = int(columns[5].text.strip())
                GoalsAgainst = int(columns[6].text.strip())
                GD = GoalsFor - GoalsAgainst
                GFG = GoalsFor/GamesPlayed
                GAG = GoalsAgainst/GamesPlayed
                GDG = GD/GamesPlayed
    #             RPT_perc = columns[7].text.strip()
    #             RW = columns[8].text.strip()
    #             RgRec = columns[9].text.strip()
    #             RgPt_perc = columns[10].text.strip()

                team_stats.extend((
                    GamesPlayed,
                    Wins,
                    Losses,
                    OvertimeLosses,
                    Points,
                    round(Points_perc,4),
                    GoalsFor,
                    GoalsAgainst,
                    GD,
                    round(GFG,2),
                    round(GAG,2),
                    round(GDG,2)
    #                 ,RPT_perc,
    #                 RW,
    #                 RgRec,
    #                 RgPt_perc
                ))

                teams = list(team_stats) #copying team_stats list into teams because ???
                break

            #Gathering Team Names by parsing through th
            for th in column_headers:
                team_names.append(column_headers[0].text.strip())
                break

    chunks = len(headers)-1
    list_of_lists = [teams[i:i + chunks] for i in range(0, len(teams), chunks)]
    team_stats.clear()

    for elem in range(len(list_of_lists)):
        list_of_lists[elem] = [team_names[elem].replace("*", "")] + list_of_lists[elem]

    #list_of_lists.insert(0,headers) #Inserting team names infront of lists with the associated stats...
                                    #...because the 2 were separate before

    return(list_of_lists) #Make list_of_lists a dict???

In [6]:
"""
- Figuring out the Playoff Teams
- Two Wild card teams from each conference are decided.

1 vs 8
2 vs 3
4 vs 7
5 vs 6

where Wild Card A (#7) pts > Wild Card B (#8) pts
And 1-3 is division 1 and 4-6 is divison 2
"""
def get_playoffs_list(list_of_lists):
    playoffs_list = (list_of_lists[0:3] + 
                    list_of_lists[8:11] + 
                    list_of_lists[16:19] + 
                    list_of_lists[24:27])

    WildCardlist =  (list_of_lists[3:8] +
                    list_of_lists[11:16] +
                    list_of_lists[19:24] + 
                    list_of_lists[27:32])

    playoffs_list.insert(6,WildCardlist.pop(WildCardlist.index(max(WildCardlist[0:11], key=lambda x: x[5]))))
    playoffs_list.insert(7,WildCardlist.pop(WildCardlist.index(max(WildCardlist[0:10], key=lambda x: x[5]))))
    playoffs_list.insert(14,WildCardlist.pop(WildCardlist.index(max(WildCardlist[8:19], key=lambda x: x[5]))))
    playoffs_list.insert(15,WildCardlist.pop(WildCardlist.index(max(WildCardlist[8:18], key=lambda x: x[5]))))

    rearranged = list(range(8,16)) + list(range(8))
    playoffs_list = [playoffs_list[elem] for elem in rearranged]
    
    return(playoffs_list)

**Main**

In [7]:
def main ():
    """ The main function is the core program code. """  
   
    #Requesting URl
    url = generate_urls() 
    soup = response_request(url)
    list_of_lists = extract_table (soup)
    playoff_list = get_playoffs_list(list_of_lists)
    #playoff_list.insert(0,headers)
    return(pd.DataFrame(playoff_list),pd.DataFrame(list_of_lists))

df1,df2 = main()

df1.columns = headers
df2.columns = headers

print("Playoff Standings")
display(df1)
print()
print("League Standings")
display(df2)


Playoff Standings


,Team,Games played,Wins,Losses,Overtime Losses,Points,Point Percentage,Goals For,Goals Against,Goal Differential,Goals For per Game,Goals Against per Game,Goal Differential per Game
0,Colorado Avalanche,82,56,19,7,119,0.7256,312,234,78,3.80,2.85,0.95
1,Minnesota Wild,82,53,22,7,113,0.6890,310,253,57,3.78,3.09,0.70
2,St. Louis Blues,82,49,22,11,109,0.6646,311,242,69,3.79,2.95,0.84
3,Calgary Flames,82,50,21,11,111,0.6768,293,208,85,3.57,2.54,1.04
4,Edmonton Oilers,82,49,27,6,104,0.6341,290,252,38,3.54,3.07,0.46
5,Los Angeles Kings,82,44,27,11,99,0.6037,239,236,3,2.91,2.88,0.04
6,Dallas Stars,82,46,30,6,98,0.5976,238,246,-8,2.90,3.00,-0.10
7,Nashville Predators,82,45,30,7,97,0.5915,266,252,14,3.24,3.07,0.17
8,Florida Panthers,82,58,18,6,122,0.7439,340,246,94,4.15,3.00,1.15
9,Toronto Maple Leafs,82,54,21,7,115,0.7012,315,253,62,3.84,3.09,0.76



League Standings


,Team,Games played,Wins,Losses,Overtime Losses,Points,Point Percentage,Goals For,Goals Against,Goal Differential,Goals For per Game,Goals Against per Game,Goal Differential per Game
0,Florida Panthers,82,58,18,6,122,0.7439,340,246,94,4.15,3.00,1.15
1,Toronto Maple Leafs,82,54,21,7,115,0.7012,315,253,62,3.84,3.09,0.76
2,Tampa Bay Lightning,82,51,23,8,110,0.6707,287,233,54,3.50,2.84,0.66
3,Boston Bruins,82,51,26,5,107,0.6524,255,220,35,3.11,2.68,0.43
4,Buffalo Sabres,82,32,39,11,75,0.4573,232,290,-58,2.83,3.54,-0.71
5,Detroit Red Wings,82,32,40,10,74,0.4512,230,312,-82,2.80,3.80,-1.00
6,Ottawa Senators,82,33,42,7,73,0.4451,227,266,-39,2.77,3.24,-0.48
7,Montreal Canadiens,82,22,49,11,55,0.3354,221,319,-98,2.70,3.89,-1.20
8,Carolina Hurricanes,82,54,20,8,116,0.7073,278,202,76,3.39,2.46,0.93
9,New York Rangers,82,52,24,6,110,0.6707,254,207,47,3.10,2.52,0.57


In [8]:
"""
Create single instance for each series instead of comparing 2 lists for each series.

Stats:
Conference Standing
Games Played

Wins (2 Points)
Losses (0 Points)
Overtime Losses (1 Point)
Points

Goals For
Goals Against
Goal Differential

5v5 Goals For
5v5 Goals Against
5v5 Goal Differential

Shot Percentage (Goals/Total Shots)
Save Percentage (Saves/Total Shots)
PDO - SPSV% (Sum of Shot Percentage and Save Percentage multiplied by 10)

Time on Attack (Avg?)

Powerplays
Power Play Success Percentage (PP Goals / Total PP)

Penalties
Penalty Kill Success Ratio (PK Success / Total Penalties)


Win Streak
Loss Streak

Fenwick Close
Team’s performance against stronger teams

Specific Game Stats
Home Advantage
LastMeetingWinner
Injuries
Corsi For percentage
"""

"""
File Format for storing Game Data:
YYYY-MM-DD-gameID-Home Team-Away Team-[winner]-A or B.csv

Label A indicates home team, B indicates Away team 
Ex: 2015-10-08-11-COL-MIN-[COL]-B.

File Format for storing Team Data:
Team Name.csv
"""

"""
Need Season data from previous 10-20 years

Classifiers: Decision-Tree, Support Vector Machine (Linear Kernel), Naive Bayes (Gaussian), Stochastic
Gradient Descent, and finally Neural Networks in NeurophStudio.

Cross Validation: k-Fold Cross Validation over 10 folds

Training NN Algorithm Types: Supervised learning, Unsupervised learning and Reinforcement learning
"""

"""
References
[0] https://webdocs.cs.ualberta.ca/~alona/dm_projs/nhl_final_report.pdf

[1] Joshua Weissbock, Forecasting Success in the National Hockey League using In-Game Statistics and Textual Data, 
Faculty of Engineering, University of Ottawa: 2014.

[2] Joshua Weissbock, Herna Viktor and Diana Inkpen, Use of Performance Metrics to Forecast Success in the National Hockey League,
Faculty of Engineering, University of Ottawa: 2013.

[3] War-on-ice.com, Team History.
www.war-on-ice.com/teambygame.html 2016. Accessed: January
2016

[4] Sandtander Meterology Group, Data Mining and
Artificial Intelligence: Bayesian and Neural Networks
http://www.meteo.unican.es/research/datamining:2010.

[5] Elnaz Davoodi and Ali Reza Khanteymoori, Horse Racing Prediction
Using Artificial Neural Networks, Mathematics and Computer Science Department, Institute for Advanced Studies in Basic Sciences:
2010.

[6] neuroph.sourceforge.net,Predicting the Result of Football Match with
Neural Networks
neuroph.sourceforge.net/tutorials/SportsPrediction/Premier League Prediction.html 2016.

[7] S. Gopal Krishna Patro and Kishore Kumar sahoe, Normalization:
A Preprocessing Stage, Department of CSE and IT, VSSUT, Burla,
India : 2010.
"""

'\nReferences\n[0] https://webdocs.cs.ualberta.ca/~alona/dm_projs/nhl_final_report.pdf\n\n[1] Joshua Weissbock, Forecasting Success in the National Hockey League using In-Game Statistics and Textual Data, \nFaculty of Engineering, University of Ottawa: 2014.\n\n[2] Joshua Weissbock, Herna Viktor and Diana Inkpen, Use of Performance Metrics to Forecast Success in the National Hockey League,\nFaculty of Engineering, University of Ottawa: 2013.\n\n[3] War-on-ice.com, Team History.\nwww.war-on-ice.com/teambygame.html 2016. Accessed: January\n2016\n\n[4] Sandtander Meterology Group, Data Mining and\nArtificial Intelligence: Bayesian and Neural Networks\nhttp://www.meteo.unican.es/research/datamining:2010.\n\n[5] Elnaz Davoodi and Ali Reza Khanteymoori, Horse Racing Prediction\nUsing Artificial Neural Networks, Mathematics and Computer Science Department, Institute for Advanced Studies in Basic Sciences:\n2010.\n\n[6] neuroph.sourceforge.net,Predicting the Result of Football Match with\nNeur

Formatting Python Code: https://stackoverflow.com/questions/44686892/loop-through-table-rows-using-beautifulsoup
Research Paper: https://webdocs.cs.ualberta.ca/~alona/dm_projs/nhl_final_report.pdf